<a href="https://colab.research.google.com/github/STLNFTART/MotorHandPro/blob/main/notebooks/experiments/03_swarm_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Swarm Simulation Analysis

Multi-agent UAV swarm coordination using Primal Logic:
- Swarm consensus
- Trust dynamics
- Neighbor graphs
- Formation control

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install numpy matplotlib pandas
    !git clone https://github.com/STLNFTART/MotorHandPro.git
    sys.path.append('/content/MotorHandPro')
else:
    sys.path.append('..' if 'notebooks' not in str(Path.cwd()) else '../..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Swarm Initialization

In [ ]:
# Swarm parameters
n_agents = 10
lambda_val = 0.16905

class SwarmAgent:
    def __init__(self, agent_id):
        self.id = agent_id
        self.position = np.random.rand(3) * 10
        self.velocity = np.zeros(3)
        self.trust = np.ones(10) * 0.5
        
    def update(self, neighbors, dt=0.01):
        # Consensus control
        consensus = np.mean([n.position for n in neighbors], axis=0)
        self.velocity = 0.5 * (consensus - self.position)
        self.position += self.velocity * dt
        
agents = [SwarmAgent(i) for i in range(n_agents)]
print(f'Initialized {n_agents} swarm agents')

## 2. Swarm Visualization

In [ ]:
# Simulate swarm
history = []
for t in range(100):
    positions = [a.position.copy() for a in agents]
    history.append(positions)
    for agent in agents:
        neighbors = [a for a in agents if a.id != agent.id]
        agent.update(neighbors)

# Plot trajectories
fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(121, projection='3d')
for i in range(n_agents):
    traj = np.array([h[i] for h in history])
    ax1.plot(traj[:, 0], traj[:, 1], traj[:, 2], alpha=0.7)
ax1.set_title('Swarm Trajectories', fontweight='bold')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

ax2 = fig.add_subplot(122)
for i in range(n_agents):
    traj = np.array([h[i] for h in history])
    dist_to_center = np.linalg.norm(traj - np.mean(traj, axis=0), axis=1)
    ax2.plot(dist_to_center, alpha=0.7)
ax2.set_title('Distance to Swarm Center', fontweight='bold')
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Distance')
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()